<a href="https://colab.research.google.com/github/ced-sys/earthquakes-and-Python/blob/main/monte_carlo_sim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install matplotlib scipy

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
import seaborn as sns

In [ ]:
class EarthquakeFaultSimulation:
    def __init__(self, fault_length=100, fault_width=20, fault_depth=5):
        """
        Initialize earthquake fault simulation parameters

        Parameters:
        fault_length: Length of fault in km
        fault_width: Width of fault in km
        fault_depth: Depth of fault in km
        """
        self.fault_length = fault_length
        self.fault_width = fault_width
        self.fault_depth = fault_depth

        # Gutenberg-Richter parameters (frequency-magnitude relationship)
        self.b_value = 1.0  # b-value (slope of magnitude-frequency relationship)
        self.a_value = 4.0  # a-value (intercept)
        self.min_magnitude = 3.0
        self.max_magnitude = 8.5

        # Temporal parameters
        self.mean_recurrence_interval = 50  # years for M>6 earthquakes
        self.time_window = 1000  # simulation time window in years

    def gutenberg_richter_magnitude(self, n_events):
        """
        Generate earthquake magnitudes using Gutenberg-Richter relationship
        """
        # Use exponential distribution for magnitude exceedance
        beta = self.b_value * np.log(10)
        magnitudes = stats.expon.rvs(scale=1/beta, size=n_events) + self.min_magnitude

        # Cap at maximum magnitude
        magnitudes = np.minimum(magnitudes, self.max_magnitude)

        return magnitudes

    def spatial_distribution(self, n_events):
        """
        Generate spatial distribution of earthquakes along fault
        """
        # Assume uniform distribution along fault length
        x_positions = np.random.uniform(0, self.fault_length, n_events)

        # Depth follows a truncated normal distribution
        depths = np.random.normal(self.fault_depth, 2, n_events)
        depths = np.clip(depths, 0.1, self.fault_depth * 2)

        # Width position (perpendicular to fault)
        y_positions = np.random.normal(0, self.fault_width/4, n_events)

        return x_positions, y_positions, depths

    def temporal_distribution(self, n_events):
        """
        Generate temporal distribution using Poisson process
        """
        # Inter-event times follow exponential distribution
        inter_event_times = np.random.exponential(self.mean_recurrence_interval, n_events)

        # Convert to cumulative time
        event_times = np.cumsum(inter_event_times)

        # Filter events within time window
        valid_events = event_times <= self.time_window

        return event_times[valid_events]

    def calculate_rupture_area(self, magnitude):
        """
        Calculate rupture area using Wells & Coppersmith (1994) scaling relationship
        """
        # Log(Area) = -3.49 + 0.91*M for strike-slip faults
        log_area = -3.49 + 0.91 * magnitude
        area = 10 ** log_area  # km²
        return area

    def calculate_peak_ground_acceleration(self, magnitude, distance):
        """
        Estimate Peak Ground Acceleration using Boore-Atkinson GMPE
        Simplified version for demonstration
        """
        # Simplified attenuation relationship
        log_pga = magnitude - 3.512 - 0.833 * np.log10(distance + 5.8)
        pga = 10 ** log_pga  # in g (gravitational acceleration)

        # Add random variability
        sigma = 0.3  # standard deviation in log space
        log_pga_with_uncertainty = log_pga + np.random.normal(0, sigma, len(distance))
        pga_with_uncertainty = 10 ** log_pga_with_uncertainty

        return pga_with_uncertainty

    def run_simulation(self, n_simulations=1000):
        """
        Run Monte Carlo simulation
        """
        results = []

        for sim in range(n_simulations):
            # Generate number of events (Poisson distributed)
            lambda_events = self.time_window / self.mean_recurrence_interval
            n_events = np.random.poisson(lambda_events * 0.1)  # Scale down for demo

            if n_events == 0:
                continue

            # Generate earthquake parameters
            magnitudes = self.gutenberg_richter_magnitude(n_events)
            x_pos, y_pos, depths = self.spatial_distribution(n_events)
            event_times = self.temporal_distribution(n_events)

            # Ensure we have the same number of events after temporal filtering
            min_events = min(len(magnitudes), len(event_times))
            magnitudes = magnitudes[:min_events]
            x_pos = x_pos[:min_events]
            y_pos = y_pos[:min_events]
            depths = depths[:min_events]
            event_times = event_times[:min_events]

            # Calculate derived parameters
            rupture_areas = self.calculate_rupture_area(magnitudes)

            # Calculate distance to a reference point (e.g., major city)
            ref_x, ref_y = 50, 0  # Reference point at fault midpoint
            distances = np.sqrt((x_pos - ref_x)**2 + (y_pos - ref_y)**2 + depths**2)

            # Calculate PGA at reference point
            pga_values = self.calculate_peak_ground_acceleration(magnitudes, distances)

            # Store results
            for i in range(min_events):
                results.append({
                    'simulation': sim,
                    'magnitude': magnitudes[i],
                    'x_position': x_pos[i],
                    'y_position': y_pos[i],
                    'depth': depths[i],
                    'time': event_times[i],
                    'rupture_area': rupture_areas[i],
                    'distance_to_ref': distances[i],
                    'pga_at_ref': pga_values[i]
                })

        return pd.DataFrame(results)

    def analyze_results(self, results_df):
        """
        Analyze and visualize simulation results
        """
        if results_df.empty:
            print("No earthquake events generated in simulation")
            return

        print("=== EARTHQUAKE SIMULATION ANALYSIS ===")
        print(f"Total simulated events: {len(results_df)}")
        print(f"Average events per simulation: {len(results_df)/results_df['simulation'].nunique():.2f}")
        print(f"\nMagnitude Statistics:")
        print(f"  Mean magnitude: {results_df['magnitude'].mean():.2f}")
        print(f"  Max magnitude: {results_df['magnitude'].max():.2f}")
        print(f"  Events M≥6.0: {len(results_df[results_df['magnitude'] >= 6.0])}")
        print(f"  Events M≥7.0: {len(results_df[results_df['magnitude'] >= 7.0])}")

        print(f"\nSpatial Distribution:")
        print(f"  Mean depth: {results_df['depth'].mean():.1f} km")
        print(f"  Fault coverage: {results_df['x_position'].std():.1f} km std dev")

        print(f"\nHazard at Reference Point:")
        print(f"  Mean PGA: {results_df['pga_at_ref'].mean():.3f} g")
        print(f"  95th percentile PGA: {results_df['pga_at_ref'].quantile(0.95):.3f} g")
        print(f"  Events with PGA > 0.1g: {len(results_df[results_df['pga_at_ref'] > 0.1])}")

        # Create visualizations
        fig, axes = plt.subplots(2, 3, figsize=(15, 10))
        fig.suptitle('Monte Carlo Earthquake Simulation Results', fontsize=16)

        # Magnitude distribution
        axes[0,0].hist(results_df['magnitude'], bins=30, alpha=0.7, edgecolor='black')
        axes[0,0].set_xlabel('Magnitude')
        axes[0,0].set_ylabel('Frequency')
        axes[0,0].set_title('Magnitude Distribution')
        axes[0,0].axvline(results_df['magnitude'].mean(), color='red', linestyle='--',
                         label=f'Mean: {results_df["magnitude"].mean():.2f}')
        axes[0,0].legend()

        # Spatial distribution
        scatter = axes[0,1].scatter(results_df['x_position'], results_df['y_position'],
                                  c=results_df['magnitude'], cmap='Reds', alpha=0.6)
        axes[0,1].set_xlabel('Position along fault (km)')
        axes[0,1].set_ylabel('Position across fault (km)')
        axes[0,1].set_title('Spatial Distribution (colored by magnitude)')
        plt.colorbar(scatter, ax=axes[0,1], label='Magnitude')

        # Temporal distribution
        axes[0,2].hist(results_df['time'], bins=30, alpha=0.7, edgecolor='black')
        axes[0,2].set_xlabel('Time (years)')
        axes[0,2].set_ylabel('Frequency')
        axes[0,2].set_title('Temporal Distribution')

        # PGA distribution
        axes[1,0].hist(np.log10(results_df['pga_at_ref']), bins=30, alpha=0.7, edgecolor='black')
        axes[1,0].set_xlabel('Log10(PGA) at Reference Point')
        axes[1,0].set_ylabel('Frequency')
        axes[1,0].set_title('Peak Ground Acceleration Distribution')

        # Magnitude vs PGA
        axes[1,1].scatter(results_df['magnitude'], np.log10(results_df['pga_at_ref']), alpha=0.5)
        axes[1,1].set_xlabel('Magnitude')
        axes[1,1].set_ylabel('Log10(PGA)')
        axes[1,1].set_title('Magnitude vs PGA Relationship')

        # Depth distribution
        axes[1,2].hist(results_df['depth'], bins=30, alpha=0.7, edgecolor='black')
        axes[1,2].set_xlabel('Depth (km)')
        axes[1,2].set_ylabel('Frequency')
        axes[1,2].set_title('Depth Distribution')

        plt.tight_layout()
        plt.show()

        return results_df

In [ ]:
if __name__=="__main__":
  #Create simulation instance
  sim=EarthquakeFaultSimulation(fault_length=100, fault_width=20, fault_depth=10)

  #Run Monte Carlo simulation
  print("Running Monte Carllo earthquake simulation...")
  results=sim.run_simulation(n_simulations=500)

  #Analyze results
  analyzed_results=sim.analyze_results(results)

  #Calculate probabilistic hazard
  print("\n=== Probabilistic Hazard Assessment ===")
  if not results.empty:
    #Annual exceedance probability for different PGA levels
    pga_levels=[0.05, 0.1, 0.2, 0.3, 0.5]
    time_span=sim.time_window

    print("Annual exceedance probabilities at reference point:")
    for pga_level in pga_levels:
      n_exceedances=len(results[results['pga_at_ref']> pga_level])
      total_events=len(results)
      if total_events>0:
        annual_prob=(n_exceedances/ len(results))* (len(results)/time_span)
        print(f"  PGA > {pga_level}g: {annual_prob:.6f}(return period: {1/annual_prob:.1f} years)"
        if annual_prob>0 else f"  PGA >{pga_level}g: 0 (no exceedances)")